## Config

In [2]:
import pandas as pd
import configparser

In [3]:
# config
config = configparser.ConfigParser()
config.read('C:/Users/USER/Documents/github_personal/study/07_AI/lightfm/core/config.ini')
config['path']['dir.model.trained']
train_dataset_dir = config['path']['dir.dataset.train']
base_path = 'C:/Users/USER/Documents/github_personal/study/07_AI/lightfm'

## Transform Dataset From .dat to .csv

In [4]:
dataset_name = 'ml-1m'
dataset_path = train_dataset_dir + '/' + dataset_name

### User

In [5]:
# user_data
data_name = 'users.dat'
data_path = base_path + '/' + dataset_path + '/' + data_name
# with open(data_path, 'r') as f:
#     lines = f.readlines()
#     lines = [line.split('::') for line in lines]

# user_df = pd.DataFrame(lines, columns=['user_idx', 'genger', 'age', 'occupatioin', 'zip_code'])
user_df = pd.read_csv(data_path, sep='::', names=['user_idx', 'gender', 'age', 'occupation', 'zip_code'])
user_df

C:\Users\USER\AppData\Local\Temp\ipykernel_5476\2009283203.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  user_df = pd.read_csv(data_path, sep='::', names=['user_idx', 'gender', 'age', 'occupation', 'zip_code'])


,user_idx,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [6]:
# 나이 변환
user_df['age'] = user_df['age'].astype('str')
user_df.loc[user_df['age'] == '1', 'age'] = '18-'
user_df.loc[user_df['age'] == '56', 'age'] = '56+'

# 직업 변환
occupation_dict = {
    0: 'other or not specified',
    1: 'academic/educator',
    2: 'artist',
    3: 'clerical/admin',
    4: 'college/grad student',
    5: 'customer service',
    6: 'doctor/health care',
    7: 'executive/managerial',
    8: 'farmer',
    9: 'homemaker',
    10: 'K-12 student',
    11: 'lawyer',
    12: 'programmer',
    13: 'retired',
    14: 'sales/marketing',
    15: 'scientist',
    16: 'self-employed',
    17: 'technician/engineer',
    18: 'tradesman/craftsman',
    19: 'unemployed',
    20: 'writer'
}
user_df['occupation'] = user_df['occupation'].map(occupation_dict)

# 저장
dataset_name = 'ml-1m-csv'
data_name = 'users.csv'
data_path = base_path + '/' + train_dataset_dir + '/' + dataset_name + '/' + data_name
user_df.to_csv(data_path)

### Movie

In [7]:
# movie_data
data_name = 'movies.dat'
data_path = base_path + '/' + dataset_path + '/' + data_name
movie_df = pd.read_csv(data_path, sep='::', names=['movie_idx', 'title', 'genres'], encoding='latin1', dtype=str)
movie_df

C:\Users\USER\AppData\Local\Temp\ipykernel_5476\3855196711.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movie_df = pd.read_csv(data_path, sep='::', names=['movie_idx', 'title', 'genres'], encoding='latin1', dtype=str)


,movie_idx,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [8]:
# movie_link
dataset = 'ml-1m'
data_name = 'links.csv'
data_path = base_path + '/' + dataset_path + '/' + data_name
movie_link_info = { info['movieId']:info['tmdbId'] for info in pd.read_csv(data_path, dtype=str).to_dict(orient='records') }
movie_df['tmdb_idx'] = movie_df['movie_idx'].map(movie_link_info)
movie_df['tmdb_idx'] = movie_df['tmdb_idx'].astype(str)

# tmdb dataset
dataset = 'tmdb'
data_name = 'movie_info.csv'
data_path = base_path + '/' + dataset_path + '/' + data_name
movie_info_df = pd.read_csv(data_path, index_col=0)
movie_info_df

# merge movie info
movie_df_custom = movie_df.copy()
movie_df_custom['release_date'] = movie_df_custom['tmdb_idx'].map(movie_info_df.set_index('id')['release_date'].to_dict())
movie_df_custom['original_language'] = movie_df_custom['tmdb_idx'].map(movie_info_df.set_index('id')['original_language'].to_dict())
movie_df_custom['adult'] = movie_df_custom['tmdb_idx'].map(movie_info_df.set_index('id')['adult'].to_dict())
movie_df_custom.drop(columns = ['tmdb_idx'], inplace=True)

# split genres
genre_dummies = movie_df_custom['genres'].str.get_dummies(sep='|')
movie_df_custom = pd.concat([movie_df_custom, genre_dummies], axis=1)

# 데이터가 null 인 항목들 제거
null_movies = movie_df_custom[movie_df_custom.isna().any(axis=1)]['movie_idx']
movie_df_custom = movie_df_custom[movie_df_custom.isna().any(axis=1) == False]
dataset_name = 'ml-1m-csv'
data_name = 'movies.csv'
data_path = base_path + '/' + train_dataset_dir + '/' + dataset_name + '/' + data_name
movie_df_custom.to_csv(data_path)

### Rating

In [9]:
# rating_data
data_name = 'ratings.dat'
data_path = base_path + '/' + dataset_path + '/' + data_name
rating_df = pd.read_csv(data_path, sep='::', names=['user_idx', 'movie_idx', 'rating', 'time'])
rating_df

C:\Users\USER\AppData\Local\Temp\ipykernel_5476\3846493279.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  rating_df = pd.read_csv(data_path, sep='::', names=['user_idx', 'movie_idx', 'rating', 'time'])


,user_idx,movie_idx,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [11]:
# remove null movies
rating_df_custom = rating_df.copy().astype(str)
rating_df_custom = rating_df_custom[rating_df_custom['movie_idx'].isin(null_movies.to_list()) != True]

# remove rating time col
rating_df_custom.drop(columns=['time'], inplace=True)

# to csv
dataset_name = 'ml-1m-csv'
data_name = 'ratings.csv'
data_path = base_path + '/' + train_dataset_dir + '/' + dataset_name + '/' + data_name
rating_df_custom.to_csv(data_path)